In [2]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

from plate import Config, Shape, get_base, get_screw_positions, get_key_positions, make_plate



Enabling jupyter_cadquery replay


In [177]:
rkn = [6,6,6,5,1]

config=Config(rkn, shape=Shape.LEAN)
kp = get_key_positions(config)

In [178]:

def make_case(config:Config) -> cq.Sketch:
    """TODO: make separate floor thickness
            change fillet parameters
            add module for leds, encoder and aviator
            add screw holes"""
    case_height = config.caseHeight
    case_gap = config.caseGap
    wall_thickness=config.wallThickness
    
    
    foot_x, foot_y = (config.columnSpacing / 2 + config.switchHoleSize, config.rowSpacing / 2 +
                          config.switchHoleSize) if config.shape == Shape.LEAN else (config.switchHoleSize, config.switchHoleSize)
    case = cq.Sketch()

    
    kp = get_key_positions(config)
    
    case = case.push(kp.values())
    case = case.rect(foot_x+(case_gap+wall_thickness)*2, foot_y+(case_gap+wall_thickness)*2).faces().clean().offset(wall_thickness).clean()#.vertices().fillet(3)
    case=cq.Workplane().placeSketch(case).extrude(case_height)

    case=case.faces("+Z").shell(-wall_thickness).edges("|Z").fillet(3)

    return case

In [179]:
case = make_case(config)

In [180]:
case

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [181]:
controllerBoxLength = 20
controllerBoxWidth=80

# find position of controller box
# add controller box corners to kp
if True:

    #x_centre = max(config.row_key_num)* config.rowSpacing*0.5
    x_centre = 0.5*(min([x[0] for x in list(kp.values())])+ max([x[0] for x in list(kp.values())]))
    
    foot_x, foot_y = (config.columnSpacing / 2 + config.switchHoleSize, config.rowSpacing / 2 +
                          config.switchHoleSize) if config.shape == Shape.LEAN else (config.switchHoleSize, config.switchHoleSize)
    case = cq.Sketch()

    
    kp = get_key_positions(config)
    
    
    points_to_push = list(kp.values())
    case = case.push(points_to_push)

    
    case = (case.rect(foot_x+(config.caseGap+wall_thickness)*2, foot_y+(config.caseGap+wall_thickness)*2)
            .faces()
            .clean()
            .offset(config.wallThickness)
            .clean())
    case=cq.Workplane().placeSketch(case).extrude(config.caseHeight)
    case = (case
            .edges('>Y and <Z')
            .workplane(centerOption="CenterOfBoundBox")
            .box(controllerBoxWidth,controllerBoxLength, config.caseHeight, centered=[True,False,False])
           )

    case=case.faces("+Z").shell(-wall_thickness).edges("|Z").fillet(3)

In [182]:
case

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [52]:
0.5*(min([x[0] for x in list(kp.values())])+ max([x[0] for x in list(kp.values())]))

47.5

In [145]:
controllerBoxLength = 20
controllerBoxWidth=80

# find position of controller box
# add controller box corners to kp
if True:

    #x_centre = max(config.row_key_num)* config.rowSpacing*0.5
    x_centre = 0.5*(min([x[0] for x in list(kp.values())])+ max([x[0] for x in list(kp.values())]))
    
    foot_x, foot_y = (config.columnSpacing / 2 + config.switchHoleSize, config.rowSpacing / 2 +
                          config.switchHoleSize) if config.shape == Shape.LEAN else (config.switchHoleSize, config.switchHoleSize)
    case = cq.Sketch()

    
    kp = get_key_positions(config)
    #points_to_push = [y[1] for y in list(filter( lambda x: x[0] not in idxs_to_drop, kp.items()))]
    points_to_push=[]
    for x in range(len(config.row_key_num)):
            points_to_push.append((x,0))
            points_to_push.append((x,config.row_key_num[x]-1))
    #points_to_push.extend([(x_centre+0.5*controllerBoxWidth,80) ,(x_centre-0.5*controllerBoxWidth,80), (x_centre,80)])
    points_to_push=[]
    for x in range(len(config.row_key_num)):
            points_to_push.append(kp[(x,0)])
            points_to_push.append(kp[(x,list(reversed(config.row_key_num))[x]-1)])
    case = case.push(points_to_push)
    #case = case.push([(0,0) ,(10,10)])
    
    case = (case.rect(foot_x+(config.caseGap+wall_thickness)*2, foot_y+(config.caseGap+wall_thickness)*2)
           .faces()
           .clean()
           .offset(config.wallThickness)
           .clean())
    case=cq.Workplane().placeSketch(case).extrude(config.caseHeight)

    #case=case.faces("+Z").shell(-wall_thickness).edges("|Z").fillet(3)
    
case

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [68]:
len(config.row_key_num), config.row_key_num

(4, [6, 6, 6, 5])

[((0, 0), (9.5, 0))]

In [77]:
kp[len(config.row_key_num)-1, 3]


(57, 57)

In [79]:
[kp[len(config.row_key_num)-1, x] for x in range( 1, config.row_key_num[0])-1]

[(0, 57), (19, 57), (38, 57), (57, 57), (76, 57), (95, 57)]

In [85]:
points_to_remove = [kp[len(config.row_key_num)-1, x] for x in range(  1,config.row_key_num[0]-1)]

In [95]:
idxs_to_drop = [(len(config.row_key_num)-1, x) for x in range(1, config.row_key_num[0]-1)]

idxs_to_drop

list(filter( lambda x: x[0] not in idxs_to_drop, kp.items()))

[((0, 0), (9.5, 0)),
 ((0, 1), (28.5, 0)),
 ((0, 2), (47.5, 0)),
 ((0, 3), (66.5, 0)),
 ((0, 4), (85.5, 0)),
 ((1, 0), (0, 19)),
 ((1, 1), (19, 19)),
 ((1, 2), (38, 19)),
 ((1, 3), (57, 19)),
 ((1, 4), (76, 19)),
 ((1, 5), (95, 19)),
 ((2, 0), (0, 38)),
 ((2, 1), (19, 38)),
 ((2, 2), (38, 38)),
 ((2, 3), (57, 38)),
 ((2, 4), (76, 38)),
 ((2, 5), (95, 38)),
 ((3, 0), (0, 57)),
 ((3, 5), (95, 57))]

In [101]:
points_to_push = [y[1] for y in list(filter( lambda x: x[0] not in idxs_to_drop, kp.items()))]
points_to_push.extend([(x_centre+0.5*controllerBoxWidth,80) ,(x_centre-0.5*controllerBoxWidth,80), (x_centre,80)])
    

In [102]:
points_to_push

[(9.5, 0),
 (28.5, 0),
 (47.5, 0),
 (66.5, 0),
 (85.5, 0),
 (0, 19),
 (19, 19),
 (38, 19),
 (57, 19),
 (76, 19),
 (95, 19),
 (0, 38),
 (19, 38),
 (38, 38),
 (57, 38),
 (76, 38),
 (95, 38),
 (0, 57),
 (95, 57),
 (87.5, 80),
 (7.5, 80),
 (47.5, 80)]

In [111]:
[ [(x,) for x in range(len(config.row_key_num))] for y in [0, config.row_key_num[x]-1]]

NameError: name 'x' is not defined

In [112]:
[[(j,i) for j in range(5)] for i in range(5)]

[[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(0, 2), (1, 2), (2, 2), (3, 2), (4, 2)],
 [(0, 3), (1, 3), (2, 3), (3, 3), (4, 3)],
 [(0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]]

In [116]:
[[(x,y) for y in [0, config.row_key_num[x]-1]] for x in range(len(config.row_key_num))]

[[(0, 0), (0, 5)], [(1, 0), (1, 5)], [(2, 0), (2, 5)], [(3, 0), (3, 4)]]

In [142]:
points_to_push=[]
for x in range(len(config.row_key_num)):
        points_to_push.append(kp[(x,0)])
        points_to_push.append(kp[(x,list(reversed(config.row_key_num))[x]-1)])

In [143]:
points_to_push

[(9.5, 0), (85.5, 0), (0, 19), (95, 19), (0, 38), (95, 38), (0, 57), (95, 57)]

In [140]:
kp

{(0, 0): (9.5, 0),
 (0, 1): (28.5, 0),
 (0, 2): (47.5, 0),
 (0, 3): (66.5, 0),
 (0, 4): (85.5, 0),
 (1, 0): (0, 19),
 (1, 1): (19, 19),
 (1, 2): (38, 19),
 (1, 3): (57, 19),
 (1, 4): (76, 19),
 (1, 5): (95, 19),
 (2, 0): (0, 38),
 (2, 1): (19, 38),
 (2, 2): (38, 38),
 (2, 3): (57, 38),
 (2, 4): (76, 38),
 (2, 5): (95, 38),
 (3, 0): (0, 57),
 (3, 1): (19, 57),
 (3, 2): (38, 57),
 (3, 3): (57, 57),
 (3, 4): (76, 57),
 (3, 5): (95, 57)}